2019 Census Data for Selected Variables - Baltimore City

In [1]:
# You may need to install some of the dependencies below using "pip install"
# pip install us

In [2]:
# pip install censusgeocode
# pip install censusdata

In [3]:
# conda install -c conda-forge cenpy

In [4]:
# conda update -n base -c defaults conda


In [5]:
# From https://cenpy-devs.github.io/cenpy/:
# Cenpy (pronounced sen-pie) is a package that automatically discovers US Census Bureau API endpoints and exposes them to Python in a consistent fashion. 
# It also provides easy-to-use access to certain well-used data products, like the American Community Survey (ACS) and 2010 Decennial Census.
# pip install cenpy

In [6]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [7]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
import censusgeocode as cg
import cenpy
import gmaps
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2019)

# Configure gmaps
#gmaps.configure(api_key=gkey)

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\libpysal\weights\util.py:23: UserWarning: geopandas not available. Some functionality will be disabled.
  warn("geopandas not available. Some functionality will be disabled.")


In [8]:
# Set this to false if you're trying to do this without an internet connection
# and data which would have been fetched from an API query will be read from cached files instead
INTERNET_IS_WORKING = True

if INTERNET_IS_WORKING:
    con = cenpy.remote.APIConnection('ACSDT5Y2019')
    variables = con.variables
else:
    variables = pd.read_csv('data/ACSDT5Y2019_variables.csv',index_col='Unnamed: 0')

# No matter which product you use, a cenpy APIConnection can show you the variables 
# which it can retrieve for you, returned as a pandas DataFrame.
print(f"ACSDT5Y2019 provides {len(variables)} variables.") # how many are there?
variables.head()

ACSDT5Y2019 provides 27080 variables.


,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN
B24022_060E,Estimate!!Total:!!Female:!!Service occupations...,SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE P...,int,B24022,0,NaN,NaN,"B24022_060EA,B24022_060M,B24022_060MA",NaN
B19001B_014E,"Estimate!!Total:!!$100,000 to $124,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 201...,int,B19001B,0,NaN,NaN,"B19001B_014EA,B19001B_014M,B19001B_014MA",NaN


In [9]:
# Comments and code in this block are from https://github.com/censusreporter/nicar20-advanced-census-python/blob/master/workshop.ipynb:
# I'll use "Nicar20" as citation for the above site from here on out
# Values for 'group' are ACS table IDs; 
# For this data, when it's N/A, it's for other kinds of API variables so won't include those:
short_vars = variables[~(variables['group'] == 'N/A')] 

# Get a list of table IDs and their titles
short_vars[['group', 'concept']].drop_duplicates().sort_values('group').head(10) 

,group,concept
B01001_012E,B01001,SEX BY AGE
B01001A_002E,B01001A,SEX BY AGE (WHITE ALONE)
B01001B_029E,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
B01001C_008E,B01001C,SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ...
B01001D_008E,B01001D,SEX BY AGE (ASIAN ALONE)
B01001E_013E,B01001E,SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ...
B01001F_001E,B01001F,SEX BY AGE (SOME OTHER RACE ALONE)
B01001G_022E,B01001G,SEX BY AGE (TWO OR MORE RACES)
B01001H_001E,B01001H,"SEX BY AGE (WHITE ALONE, NOT HISPANIC OR LATINO)"
B01001I_027E,B01001I,SEX BY AGE (HISPANIC OR LATINO)


In [10]:
# (From Nicar20)
# Use when you know which group but still need specific API variable codes
# "attributes" column shows related variables you can request. The one that ends with M is the margin of error, and since we want to be responsible when we aggregate data, we'll be sure to aggregate the error as well. 
# Just fyi, The other two which end with A are "annotations." 
short_vars[short_vars['group'] == 'B01001'][['label','attributes']].sort_index() 

,label,attributes
B01001_001E,Estimate!!Total:,"B01001_001EA,B01001_001M,B01001_001MA"
B01001_002E,Estimate!!Total:!!Male:,"B01001_002EA,B01001_002M,B01001_002MA"
B01001_003E,Estimate!!Total:!!Male:!!Under 5 years,"B01001_003EA,B01001_003M,B01001_003MA"
B01001_004E,Estimate!!Total:!!Male:!!5 to 9 years,"B01001_004EA,B01001_004M,B01001_004MA"
B01001_005E,Estimate!!Total:!!Male:!!10 to 14 years,"B01001_005EA,B01001_005M,B01001_005MA"
B01001_006E,Estimate!!Total:!!Male:!!15 to 17 years,"B01001_006EA,B01001_006M,B01001_006MA"
B01001_007E,Estimate!!Total:!!Male:!!18 and 19 years,"B01001_007EA,B01001_007M,B01001_007MA"
B01001_008E,Estimate!!Total:!!Male:!!20 years,"B01001_008EA,B01001_008M,B01001_008MA"
B01001_009E,Estimate!!Total:!!Male:!!21 years,"B01001_009EA,B01001_009M,B01001_009MA"
B01001_010E,Estimate!!Total:!!Male:!!22 to 24 years,"B01001_010EA,B01001_010M,B01001_010MA"


In [11]:
# search function
# sample = censusdata.search('acs5', 2019, 'concept', 'age ')
# print(sample)

#  to see list of all tables in the ACS5:  c.acs5.tables()

In [12]:
censusdata.printtable(censusdata.censustable('acs5', 2019, 'B08604'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08604_001E  | WORKER POPULATION FOR WORKPLAC | !! Estimate Total:                                       | int  
-------------------------------------------------------------------------------------------------------------------


In [13]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
# The state FIPS code for MD is 24 and the FIPS code for Balt City is 510; * is to pull data for all census tracts in the 510 FIPs
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_005E",              
                          "B02001_008E",
                          "B03001_003E",
                          "B25008_002E",
                          "B28007_001E",
                          "B25003_002E",                                    
                          "B25003_003E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
census_pd = pd.DataFrame(census_data)
census_pd.head()      

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_005E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,0.0,0.0,0.0,306.0,1218.0,119.0,622.0,24,510,260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,31.0,99.0,88.0,1030.0,1808.0,539.0,443.0,24,510,230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,60.0,0.0,94.0,3674.0,3812.0,1384.0,323.0,24,510,260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,0.0,29.0,66.0,164.0,716.0,88.0,368.0,24,510,260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,35.0,10.0,970.0,751.0,1875.0,415.0,580.0,24,510,260800


In [14]:
# Pull values from the ACS 5 yr. census codes/ variables listed and save them in the variable "age_data"

age_data = c.acs5.state_county_tract(("NAME", "B01001_003E",                                    
                          "B01001_004E",
                          "B01001_005E",
                          "B01001_006E",
                          "B01001_007E",
                          "B01001_008E",
                          "B01001_009E",
                          "B01001_010E",
                          "B01001_011E",
                          "B01001_012E",
                          "B01001_013E",
                          "B01001_014E",
                          "B01001_015E",
                          "B01001_016E",            
                          "B01001_017E",
                          "B01001_018E",
                          "B01001_019E",
                          "B01001_020E",
                          "B01001_021E",
                          "B01001_022E",
                          "B01001_023E",
                          "B01001_024E",
                          "B01001_025E",
                          
                          "B01001_027E",
                          "B01001_028E",
                          "B01001_029E",
                          "B01001_030E",
                          "B01001_031E",
                          "B01001_032E",
                          "B01001_033E",
                          "B01001_034E",
                          "B01001_035E",
                          "B01001_036E",
                          "B01001_037E",
                          "B01001_038E",
                          "B01001_039E",            
                          "B01001_040E",
                          "B01001_041E",
                          "B01001_042E",
                          "B01001_043E",
                          "B01001_044E",
                          "B01001_045E",
                          "B01001_046E",
                          "B01001_047E",
                          "B01001_048E",
                          "B01001_049E"),          
                                     
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
age_pd = pd.DataFrame(age_data)
age_pd.head()      

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,41.0,13.0,17.0,5.0,26.0,0.0,0.0,24,510,260403
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,15.0,22.0,18.0,11.0,23.0,11.0,32.0,24,510,230200
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,147.0,116.0,49.0,64.0,57.0,23.0,45.0,24,510,260102
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,18.0,20.0,6.0,0.0,7.0,11.0,0.0,24,510,260303
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,33.0,14.0,9.0,28.0,34.0,32.0,25.0,24,510,260800


In [15]:
# Sum columns of age groups that are under 18 years old for male and female and add new column "Pop. <18 years"
columns_under18 = ["B01001_003E",                                    
                   "B01001_004E",
                   "B01001_005E",
                   "B01001_006E",
                   "B01001_027E",
                   "B01001_028E",
                   "B01001_029E",
                   "B01001_030E"]
age_pd['Pop. <18 years']= age_pd[columns_under18].sum(axis=1)
age_pd.head()

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,13.0,17.0,5.0,26.0,0.0,0.0,24,510,260403,586.0
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,22.0,18.0,11.0,23.0,11.0,32.0,24,510,230200,174.0
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,116.0,49.0,64.0,57.0,23.0,45.0,24,510,260102,1086.0
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,20.0,6.0,0.0,7.0,11.0,0.0,24,510,260303,266.0
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,14.0,9.0,28.0,34.0,32.0,25.0,24,510,260800,456.0


In [16]:
# Sum columns of age groups that are 18-64 years old (working age) for male and female and add new column "Pop. working age"
columns_working_age = [                                    
                   "B01001_007E",
                   "B01001_008E",
                   "B01001_009E",
                   "B01001_010E",
                   "B01001_011E",
                   "B01001_012E",   
                   "B01001_013E",
                   "B01001_014E",    
                   "B01001_015E",    
                   "B01001_016E",   
                   "B01001_017E", 
                   "B01001_018E",    
                   "B01001_019E",    
                   "B01001_031E",
                   "B01001_032E",
                   "B01001_033E",
                   "B01001_034E",
                   "B01001_035E",   
                   "B01001_036E",
                   "B01001_037E",    
                   "B01001_038E",    
                   "B01001_039E",   
                   "B01001_040E", 
                   "B01001_041E",    
                   "B01001_042E",    
                   "B01001_043E"]    
                           
age_pd['Pop. working age']= age_pd[columns_working_age].sum(axis=1)
age_pd.head()         
            
               


,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,17.0,5.0,26.0,0.0,0.0,24,510,260403,586.0,1045.0
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,18.0,11.0,23.0,11.0,32.0,24,510,230200,174.0,1580.0
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,49.0,64.0,57.0,23.0,45.0,24,510,260102,1086.0,3057.0
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,6.0,0.0,7.0,11.0,0.0,24,510,260303,266.0,621.0
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,9.0,28.0,34.0,32.0,25.0,24,510,260800,456.0,1630.0


In [17]:
# Sum columns of age groups that are 65+ years old for male and female and add new column "Pop. 65+ years"
columns_senior = ["B01001_020E",
                  "B01001_021E",
                  "B01001_022E",
                  "B01001_023E",                                    
                  "B01001_024E",
                  "B01001_025E",
                  "B01001_044E",
                  "B01001_045E",
                  "B01001_046E",
                  "B01001_047E",                                    
                  "B01001_048E",
                  "B01001_049E"]               
                          
age_pd['Pop. 65+ years']= age_pd[columns_senior].sum(axis=1)
age_pd.head()                           

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,"Census Tract 2604.03, Baltimore city, Maryland",51.0,164.0,114.0,22.0,12.0,0.0,13.0,18.0,52.0,...,5.0,26.0,0.0,0.0,24,510,260403,586.0,1045.0,133.0
1,"Census Tract 2302, Baltimore city, Maryland",52.0,0.0,15.0,14.0,0.0,0.0,0.0,107.0,163.0,...,11.0,23.0,11.0,32.0,24,510,230200,174.0,1580.0,228.0
2,"Census Tract 2601.02, Baltimore city, Maryland",58.0,157.0,194.0,142.0,61.0,50.0,0.0,36.0,97.0,...,64.0,57.0,23.0,45.0,24,510,260102,1086.0,3057.0,632.0
3,"Census Tract 2603.03, Baltimore city, Maryland",66.0,49.0,0.0,11.0,0.0,23.0,0.0,23.0,20.0,...,0.0,7.0,11.0,0.0,24,510,260303,266.0,621.0,82.0
4,"Census Tract 2608, Baltimore city, Maryland",79.0,53.0,0.0,3.0,10.0,0.0,16.0,58.0,73.0,...,28.0,34.0,32.0,25.0,24,510,260800,456.0,1630.0,246.0


In [18]:
age_final = age_pd[[ "tract", "Pop. <18 years", "Pop. working age", "Pop. 65+ years"]]
age_final.head()

,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,260403,586.0,1045.0,133.0
1,230200,174.0,1580.0,228.0
2,260102,1086.0,3057.0,632.0
3,260303,266.0,621.0,82.0
4,260800,456.0,1630.0,246.0


In [19]:
age_final = age_final.rename(columns={"tract": "Census_tract"})
age_final

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,260403,586.0,1045.0,133.0
1,230200,174.0,1580.0,228.0
2,260102,1086.0,3057.0,632.0
3,260303,266.0,621.0,82.0
4,260800,456.0,1630.0,246.0
...,...,...,...,...
195,220100,386.0,3241.0,820.0
196,230300,21.0,1298.0,89.0
197,250207,724.0,1401.0,212.0
198,250303,521.0,1433.0,275.0


In [20]:
# Did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state,county
0,"Baltimore city, Maryland",383366.0,24,510


In [21]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract code
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract,GEOID
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,0.0,0.0,306.0,1218.0,119.0,622.0,24,510,260403,24510260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,99.0,88.0,1030.0,1808.0,539.0,443.0,24,510,230200,24510230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,0.0,94.0,3674.0,3812.0,1384.0,323.0,24,510,260102,24510260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,29.0,66.0,164.0,716.0,88.0,368.0,24,510,260303,24510260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,10.0,970.0,751.0,1875.0,415.0,580.0,24,510,260800,24510260800


In [22]:
# "Number of rows" = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 23)


In [23]:
# remove extraneous column "tract"
census_pd = census_pd.drop(["tract"], axis=1)
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_005E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,GEOID
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,0.0,0.0,0.0,306.0,1218.0,119.0,622.0,24,510,24510260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,31.0,99.0,88.0,1030.0,1808.0,539.0,443.0,24,510,24510230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,60.0,0.0,94.0,3674.0,3812.0,1384.0,323.0,24,510,24510260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,0.0,29.0,66.0,164.0,716.0,88.0,368.0,24,510,24510260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,35.0,10.0,970.0,751.0,1875.0,415.0,580.0,24,510,24510260800


In [24]:
# GIS Analyst, Patrick, provided a csv (created from ARcGIS) that contains a key to match Baltimore's commercial corridors 
# with specific GEOIDS 

# Store filepath in a variable
corridor_key = "./CSVs/corr_key.csv"

# Read the file with the pandas library
corr_key_df = pd.read_csv(corridor_key)
corr_key_df.dtypes


GEOID        int64
Corridor    object
dtype: object

In [25]:
corr_key_df

,GEOID,Corridor
0,24510260403,Highlandtown
1,24510230200,Hamilton Lauraville
2,24510260102,Pimlico
3,24510260303,Hamilton Lauraville
4,24510260800,Highlandtown
...,...,...
195,24510220100,NaN
196,24510230300,NaN
197,24510250207,NaN
198,24510250303,NaN


In [26]:
# combine age data with rest of census data
census_joined = pd.concat([age_final, census_pd], axis="columns")


census_joined.shape

(200, 26)

In [27]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's understandable
census_joined = census_joined.rename(columns={"B01003_001E": "Population",
                                      "tract": "Census Tract",        
                                      "B01002_001E": "Median age",
                                      "B19013_001E": "Median household income",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_005E": "Pop. Asian",        
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"})

# Add a new column for poverty rate (Poverty Count / Population)
census_joined["Poverty rate"] = 100 * \
    census_joined["Poverty count"].astype(
        int) / census_joined["Population"].astype(int)

# Add a new column for unemployment rate (Employment Count / Population)
census_joined["Unemployment rate"] = 100 * \
    census_joined["Unemployment count"].astype(
        int) / census_joined["Population"].astype(int)
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,B28007_001E,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate
0,260403,586.0,1045.0,133.0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,...,0.0,306.0,1218.0,119.0,622.0,24,510,24510260403,24.716553,1.927438
1,230200,174.0,1580.0,228.0,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,...,88.0,1030.0,1808.0,539.0,443.0,24,510,24510230200,7.618567,0.857719
2,260102,1086.0,3057.0,632.0,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,...,94.0,3674.0,3812.0,1384.0,323.0,24,510,24510260102,9.821990,3.162304
3,260303,266.0,621.0,82.0,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,...,66.0,164.0,716.0,88.0,368.0,24,510,24510260303,38.906089,2.063983
4,260800,456.0,1630.0,246.0,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,...,970.0,751.0,1875.0,415.0,580.0,24,510,24510260800,20.626072,1.629503


In [28]:
# Add in home ownership rate (# owner-occupied units / # of occupied housing units)
# sum 2 columns: total owner-occupied units + total renter-occupied units to create additional column "Total occupied units" 
sum_column = census_joined['Total owner-occupied units'] + census_joined['Total renter-occupied units']
census_joined["Total occupied units"] = sum_column

In [29]:
census_joined["Home ownership rate"] = 100 * \
    census_joined["Total owner-occupied units"].astype(
        int) / census_joined["Total occupied units"].astype(
        int) 

census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,B28007_001E,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,260403,586.0,1045.0,133.0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,...,1218.0,119.0,622.0,24,510,24510260403,24.716553,1.927438,741.0,16.059379
1,230200,174.0,1580.0,228.0,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,...,1808.0,539.0,443.0,24,510,24510230200,7.618567,0.857719,982.0,54.887984
2,260102,1086.0,3057.0,632.0,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,...,3812.0,1384.0,323.0,24,510,24510260102,9.821990,3.162304,1707.0,81.077914
3,260303,266.0,621.0,82.0,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,...,716.0,88.0,368.0,24,510,24510260303,38.906089,2.063983,456.0,19.298246
4,260800,456.0,1630.0,246.0,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,...,1875.0,415.0,580.0,24,510,24510260800,20.626072,1.629503,995.0,41.708543


In [30]:
# Round the home ownership rate to one decimal point; using "float" instead of "int" because want to use decimal points
census_joined["Home ownership rate"] = census_joined["Home ownership rate"].astype(float).round(1)


In [31]:
census_joined["Poverty rate"] = census_joined["Poverty rate"].astype(float).round(1)


In [32]:
census_joined["Unemployment rate"] = census_joined["Unemployment rate"].astype(float).round(1)
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,B28007_001E,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,260403,586.0,1045.0,133.0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,...,1218.0,119.0,622.0,24,510,24510260403,24.7,1.9,741.0,16.1
1,230200,174.0,1580.0,228.0,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,...,1808.0,539.0,443.0,24,510,24510230200,7.6,0.9,982.0,54.9
2,260102,1086.0,3057.0,632.0,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,...,3812.0,1384.0,323.0,24,510,24510260102,9.8,3.2,1707.0,81.1
3,260303,266.0,621.0,82.0,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,...,716.0,88.0,368.0,24,510,24510260303,38.9,2.1,456.0,19.3
4,260800,456.0,1630.0,246.0,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,...,1875.0,415.0,580.0,24,510,24510260800,20.6,1.6,995.0,41.7


In [33]:
# Calculate population density  see: https://www.census.gov/quickfacts/fact/note/US/LND110210
# Density is expressed as "population per square mile(kilometer)"
# Divide total population (or # of housing units)/ by land area of the entity measured in square miles

In [34]:
census_joined.count()

Census_tract                                      200
Pop. <18 years                                    200
Pop. working age                                  200
Pop. 65+ years                                    200
Name                                              200
Median household income                           200
Population                                        200
Median age                                        200
Per capita income                                 200
Poverty count                                     200
Unemployment count                                200
# employed, age 16+                               200
# persons age 25+ graduated high school           200
# persons age 25+ with Bachelor's degree          200
Pop. white                                        200
Pop. Black                                        200
Pop. Asian                                        200
Pop. 2 or more races                              200
Pop. Hispanic origin        

In [35]:
census_joined.dtypes

Census_tract                                       object
Pop. <18 years                                    float64
Pop. working age                                  float64
Pop. 65+ years                                    float64
Name                                               object
Median household income                           float64
Population                                        float64
Median age                                        float64
Per capita income                                 float64
Poverty count                                     float64
Unemployment count                                float64
# employed, age 16+                               float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. Asian                                        float64
Pop. 2 or more

In [36]:
# Remove "Name" column because it's extraneous
census_joined = census_joined.drop(["Name"], axis=1)

census_joined.dtypes

Census_tract                                       object
Pop. <18 years                                    float64
Pop. working age                                  float64
Pop. 65+ years                                    float64
Median household income                           float64
Population                                        float64
Median age                                        float64
Per capita income                                 float64
Poverty count                                     float64
Unemployment count                                float64
# employed, age 16+                               float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. Asian                                        float64
Pop. 2 or more races                              float64
Pop. Hispanic 

In [37]:
# Split the "Name" column into 3 separate columns: "Census_Tract", "County", "State"
# census_joined[['Census_Tract', "County", "State"]]= census_joined['Name'].str.split(",", n=3, expand=True)
# census_joined.head()

In [38]:
# List the columns in the census_joined dataframe
census_joined.columns

Index(['Census_tract', 'Pop. <18 years', 'Pop. working age', 'Pop. 65+ years',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Unemployment count',
       '# employed, age 16+', '# persons age 25+ graduated high school',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. 2 or more races', 'Pop. Hispanic origin',
       'Total pop. in occupied housing units by tenure', 'B28007_001E',
       'Total owner-occupied units', 'Total renter-occupied units', 'State',
       'county', 'GEOID', 'Poverty rate', 'Unemployment rate',
       'Total occupied units', 'Home ownership rate'],
      dtype='object')

In [39]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_joined["Census_tract"].unique())
tract_count

200

In [40]:
# make sure that the GEOID is the same data type in each of the dataframes to be merged, by using .astype
#(int64 is a 64-bit integer, refers to how much storage needed for this datapoint)
census_joined["GEOID"] = census_joined["GEOID"].astype('int64')


In [41]:
census_joined.dtypes

Census_tract                                       object
Pop. <18 years                                    float64
Pop. working age                                  float64
Pop. 65+ years                                    float64
Median household income                           float64
Population                                        float64
Median age                                        float64
Per capita income                                 float64
Poverty count                                     float64
Unemployment count                                float64
# employed, age 16+                               float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. Asian                                        float64
Pop. 2 or more races                              float64
Pop. Hispanic 

In [42]:
corr_key_df["GEOID"] = corr_key_df["GEOID"].astype('int64')

In [43]:
corr_key_df.dtypes

GEOID        int64
Corridor    object
dtype: object

In [44]:
# merge the census_joined dataframe with the corr_key_df dataframe on the common column "GEOID"

corridors_df = pd.merge(
    census_joined, corr_key_df, on="GEOID")

# remove any columns with NaN ("Not a Number") - used for missing values, by using .dropna()
corridors_df = corridors_df.dropna()


In [45]:
# reset the index numbers for the dataframe (the first column)
corridors_df = corridors_df.reset_index(drop=True)
corridors_df.head()
# remove the "county" column as it is not needed
#corridors_df = corridors_df.drop(["county"], axis=1)


,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,...,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate,Corridor
0,260403,586.0,1045.0,133.0,31625.0,1764.0,27.9,21744.0,436.0,34.0,...,119.0,622.0,24,510,24510260403,24.7,1.9,741.0,16.1,Highlandtown
1,230200,174.0,1580.0,228.0,107391.0,1982.0,30.7,63973.0,151.0,17.0,...,539.0,443.0,24,510,24510230200,7.6,0.9,982.0,54.9,Hamilton Lauraville
2,260102,1086.0,3057.0,632.0,62586.0,4775.0,41.3,25903.0,469.0,151.0,...,1384.0,323.0,24,510,24510260102,9.8,3.2,1707.0,81.1,Pimlico
3,260303,266.0,621.0,82.0,17000.0,969.0,37.0,19562.0,377.0,20.0,...,88.0,368.0,24,510,24510260303,38.9,2.1,456.0,19.3,Hamilton Lauraville
4,260800,456.0,1630.0,246.0,40795.0,2332.0,36.1,26895.0,481.0,38.0,...,415.0,580.0,24,510,24510260800,20.6,1.6,995.0,41.7,Highlandtown


In [46]:
# Change order of columns in DataFrame by using double brackets
# Notice that columns such as "state" and "county" were not included in the code below and they are no longer included in the dataframe (they're extraneous, we don't need them)
corridors_df = corridors_df[["Corridor", "Census_tract", "GEOID", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin", 
                      "Pop. Asian","Total pop. in occupied housing units by tenure", "Total owner-occupied units", "Total renter-occupied units",
                      "Pop. <18 years", "Pop. working age", "Pop. 65+ years"]]

corridors_df.head()

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Highlandtown,260403,24510260403,1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,...,1741.0,0.0,0.0,0.0,306.0,119.0,622.0,586.0,1045.0,133.0
1,Hamilton Lauraville,230200,24510230200,1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,...,169.0,99.0,88.0,31.0,1030.0,539.0,443.0,174.0,1580.0,228.0
2,Pimlico,260102,24510260102,4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,...,4254.0,0.0,94.0,60.0,3674.0,1384.0,323.0,1086.0,3057.0,632.0
3,Hamilton Lauraville,260303,24510260303,969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,...,626.0,29.0,66.0,0.0,164.0,88.0,368.0,266.0,621.0,82.0
4,Highlandtown,260800,24510260800,2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,...,310.0,10.0,970.0,35.0,751.0,415.0,580.0,456.0,1630.0,246.0


In [47]:
# sort the column "Census_tract" so that it is easier to compare with other years that we are going to pull in another notebook
census_2019_FINAL =corridors_df.sort_values("Census_tract")
census_2019_FINAL = census_2019_FINAL.reset_index(drop=True)
census_2019_FINAL

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Highlandtown,010200,24510010200,2858.0,106250.0,57925.0,133.0,4.7,1.3,2086.0,...,128.0,72.0,92.0,126.0,2331.0,1118.0,293.0,324.0,2308.0,226.0
1,Brooklyn,020300,24510020300,3761.0,108026.0,83545.0,342.0,9.1,0.9,2985.0,...,138.0,224.0,263.0,308.0,1376.0,697.0,1455.0,214.0,3144.0,403.0
2,E Monument St,040100,24510040100,5051.0,64016.0,50026.0,626.0,12.4,1.9,3496.0,...,1532.0,298.0,501.0,866.0,322.0,230.0,2854.0,437.0,4444.0,170.0
3,Hamilton Lauraville,040200,24510040200,984.0,43417.0,19148.0,93.0,9.5,1.7,440.0,...,338.0,68.0,34.0,129.0,0.0,0.0,261.0,22.0,943.0,19.0
4,E Monument St,060200,24510060200,3037.0,69034.0,40668.0,1026.0,33.8,5.1,1516.0,...,1298.0,84.0,334.0,109.0,1547.0,578.0,505.0,771.0,2116.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Waverly,280302,24510280302,2538.0,44423.0,24760.0,519.0,20.4,4.9,1179.0,...,2201.0,209.0,15.0,12.0,834.0,314.0,708.0,606.0,1677.0,255.0
121,Pimlico,280401,24510280401,3269.0,46493.0,31029.0,350.0,10.7,2.0,1635.0,...,2669.0,111.0,23.0,16.0,2351.0,972.0,501.0,583.0,2099.0,587.0
122,Pigtown,280402,24510280402,1632.0,48750.0,23395.0,262.0,16.1,5.2,662.0,...,1601.0,1.0,28.0,0.0,1269.0,512.0,105.0,316.0,967.0,349.0
123,Belair Rd,280403,24510280403,5468.0,57973.0,34384.0,263.0,4.8,1.7,3032.0,...,4033.0,100.0,63.0,32.0,2692.0,898.0,1307.0,1247.0,3677.0,544.0


In [48]:
# Export file as a CSV, without the Pandas index, but with the header
# Do not run this last code block until you have all previous code blocks in their final form:

census_2019_FINAL.to_csv("CommCorr_Census_Stats_2019.csv", index = False, header=True)

In [49]:
census_2019_formatted = census_2019_FINAL
census_2019_formatted.head()

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Highlandtown,010200,24510010200,2858.0,106250.0,57925.0,133.0,4.7,1.3,2086.0,...,128.0,72.0,92.0,126.0,2331.0,1118.0,293.0,324.0,2308.0,226.0
1,Brooklyn,020300,24510020300,3761.0,108026.0,83545.0,342.0,9.1,0.9,2985.0,...,138.0,224.0,263.0,308.0,1376.0,697.0,1455.0,214.0,3144.0,403.0
2,E Monument St,040100,24510040100,5051.0,64016.0,50026.0,626.0,12.4,1.9,3496.0,...,1532.0,298.0,501.0,866.0,322.0,230.0,2854.0,437.0,4444.0,170.0
3,Hamilton Lauraville,040200,24510040200,984.0,43417.0,19148.0,93.0,9.5,1.7,440.0,...,338.0,68.0,34.0,129.0,0.0,0.0,261.0,22.0,943.0,19.0
4,E Monument St,060200,24510060200,3037.0,69034.0,40668.0,1026.0,33.8,5.1,1516.0,...,1298.0,84.0,334.0,109.0,1547.0,578.0,505.0,771.0,2116.0,150.0


In [50]:
# Use .map to format columns (helpful resource for this: https://towardsdatascience.com/apply-thousand-separator-and-other-formatting-to-pandas-dataframe-45f2f4c7ab01)
# Note: once you format values in a column, they are changed to strings (see cell below to see data types of each column)
# I will use the census_2017_FINAL dataframe to use for analysis as needed (can do calculations with number data types but not strings)
# You may need to restart the kernel after you format
census_2019_formatted["Median household income"] = census_2019_FINAL["Median household income"].map("${:.2f}".format)
census_2019_formatted["Per capita income"] = census_2019_FINAL["Per capita income"].map("${:.2f}".format)
census_2019_formatted["Population"] = census_2019_formatted["Population"].map("{:,.0f}".format)
census_2019_formatted["Poverty count"] = census_2019_formatted["Poverty count"].map("{:,.0f}".format)
census_2019_formatted["Poverty rate"] = census_2019_formatted["Poverty rate"].map("{:.2%}".format)
census_2019_formatted["Unemployment rate"] = census_2019_formatted["Unemployment rate"].map("{:.2%}".format)

census_2019_formatted = census_2019_formatted.reset_index(drop=True)
census_2019_formatted.head()

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Highlandtown,010200,24510010200,"2,858",$106250.00,$57925.00,133,470.00%,130.00%,2086.0,...,128.0,72.0,92.0,126.0,2331.0,1118.0,293.0,324.0,2308.0,226.0
1,Brooklyn,020300,24510020300,"3,761",$108026.00,$83545.00,342,910.00%,90.00%,2985.0,...,138.0,224.0,263.0,308.0,1376.0,697.0,1455.0,214.0,3144.0,403.0
2,E Monument St,040100,24510040100,"5,051",$64016.00,$50026.00,626,1240.00%,190.00%,3496.0,...,1532.0,298.0,501.0,866.0,322.0,230.0,2854.0,437.0,4444.0,170.0
3,Hamilton Lauraville,040200,24510040200,984,$43417.00,$19148.00,93,950.00%,170.00%,440.0,...,338.0,68.0,34.0,129.0,0.0,0.0,261.0,22.0,943.0,19.0
4,E Monument St,060200,24510060200,"3,037",$69034.00,$40668.00,"1,026",3380.00%,510.00%,1516.0,...,1298.0,84.0,334.0,109.0,1547.0,578.0,505.0,771.0,2116.0,150.0


In [51]:
census_2019_formatted.dtypes

Corridor                                           object
Census_tract                                       object
GEOID                                               int64
Population                                         object
Median household income                            object
Per capita income                                  object
Poverty count                                      object
Poverty rate                                       object
Unemployment rate                                  object
# employed, age 16+                               float64
Unemployment count                                float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Median age                                        float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. 2 or more races                              float64
Pop. Hispanic 